# [75.06 / 95.58] Organización de Datos <br> Trabajo Práctico 2: Machine Learning

# Investigación Previa

**Grupo 30: Datatouille**

**http://fdelmazo.github.io/7506-Datos/**

Este notebook pretende, con mucha ayuda del trabajo realizado en el [TP1](https://fdelmazo.github.io/7506-Datos/TP1/TP1.html), encontrar y recopilar información a utilizar en el resto del trabajo.

In [ ]:
import pandas as pd
import numpy as np
from ggplot import * # pip install ggplot # https://stackoverflow.com/a/50607072/10728610
import time
from sklearn.manifold import TSNE
import sklearn.cluster as cluster
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# !unzip -q ../TP1/data/events.zip -d ../TP1/data
df_tp1 = pd.read_csv('../TP1/data/events.csv', low_memory=False)
df_tp2 = pd.read_csv('./data/events_up_to_01062018.csv', low_memory=False)
df_labels = pd.read_csv('./data/labels_training_set.csv', low_memory=False)

## Comparación TP1 y TP2

In [ ]:
def labelize(df):
    df_tmp = df
    df_tmp['timestamp'] = pd.to_datetime(df_tmp['timestamp'])
    
    df_tmp['is_useful_conversion'] = (df_tmp['event'] == 'conversion') \
        & (df_tmp['timestamp'] > pd.to_datetime('2018-06-01'))
    
    df_tmp = df_tmp[['person', 'is_useful_conversion']]
    gb = df_tmp.groupby('person')
    df_tmp = gb.sum()
    df_tmp.columns = ['label']
    df_tmp['label'] = df_tmp['label'].astype('bool')
    df_tmp['label'] = df_tmp['label'].astype('int')
    
    return df_tmp

In [ ]:
df_tp1_labels = labelize(df_tp1)

display(df_tp1_labels.head())
display("Usuarios en TP1: " + str(len(df_tp1_labels)))
display("Usuarios que no convirtieron: " + str(len(df_tp1_labels[df_tp1_labels['label'] == 0])))
display("Usuarios que convirtieron: " + str(len(df_tp1_labels[df_tp1_labels['label'] == 1])))

In [ ]:
df_tp2_labels = labelize(df_tp2)

display(df_tp2_labels.head())
display("Usuarios en TP2: " + str(len(df_tp2_labels)))
display("Usuarios que no convirtieron: " + str(len(df_tp2_labels[df_tp2_labels['label'] == 0])))
display("Usuarios que convirtieron: " + str(len(df_tp2_labels[df_tp2_labels['label'] == 1])))

In [ ]:
# Chequeando que todos los usuarios tienen actividad después de 2018-06-01
df_tmp = df_tp1
df_tmp['timestamp'] = pd.to_datetime(df_tmp['timestamp'])
df_tmp = df_tmp.loc[df_tmp['timestamp'] > pd.to_datetime('2018-06-01')]

display(len(df_tp1['person'].unique()))
display(len(df_tmp['person'].unique()))
display(len(df_tmp[df_tmp['timestamp'] < pd.to_datetime('2018-06-16')]['person'].unique()))

In [ ]:
display("Usuarios en set de entrenamiento: " + str(len(df_labels)))
display("Usuarios que no convirtieron: " + str(len(df_labels[df_labels['label'] == 0])))
display("Usuarios que convirtieron: " + str(len(df_labels[df_labels['label'] == 1])))

**Información Recopilada**

1. No se repiten usuarios en los datasets.
2. En el **primer** dataset (TP1) hay 27624 usuarios.
3. De los cuales 13967 tuvieron actividad en Junio.
4. Entre el 1 y el 15 (inclusive) de Junio 82 usuarios compraron productos.
2. En el **segundo** dataset hay 19414 usuarios.
3. De los cuales 980 compraron en Junio.

**Conclusión**

Hacer un merge de los datos del TP1 con los del TP2 presentaría un *skewness* en el set de datos, por la despreciabilidad de estos. Es mejor no hacerlo.

## Visualización con reducción de dimensiones

Usamos TSNE

In [ ]:
df_users = pd.read_csv('data/user-features.csv',low_memory=False).set_index('person')

In [ ]:
time_start = time.time()
tsne = TSNE(n_components=2, verbose=1, perplexity=3, n_iter=300)
tsne_results = tsne.fit_transform(df_users.values)

print('t-SNE done! Time elapsed: {} seconds'.format(time.time()-time_start))

In [ ]:
df_tsne = df_users.copy()
df_tsne['x-tsne'] = tsne_results[:,0]
df_tsne['y-tsne'] = tsne_results[:,1]

chart = ggplot(df_tsne, aes(x='x-tsne', y='y-tsne', color='has_conversion') ) \
        + geom_point(size=70,alpha=0.1) \
        + ggtitle("tSNE dimensions colored by has_conversion")
chart

Ahora ploteamos con K-means

In [ ]:
sns.set_context('poster')
sns.set_color_codes()
plot_kwds = {'alpha' : 0.25, 's' : 80, 'linewidths':0}

In [ ]:
plt.scatter(df_users.T[], df_users.T[1], c='b', **plot_kwds)
frame = plt.gca()
frame.axes.get_xaxis().set_visible(False)
frame.axes.get_yaxis().set_visible(False)

In [ ]:
def plot_clusters(data, algorithm, args, kwds):
    start_time = time.time()
    labels = algorithm(*args, **kwds).fit_predict(data)
    end_time = time.time()
    palette = sns.color_palette('deep', np.unique(labels).max() + 1)
    colors = [palette[x] if x >= 0 else (0.0, 0.0, 0.0) for x in labels]
    plt.scatter(data.T[0], data.T[1], c=colors, **plot_kwds)
    frame = plt.gca()
    frame.axes.get_xaxis().set_visible(False)
    frame.axes.get_yaxis().set_visible(False)
    plt.title('Clusters found by {}'.format(str(algorithm.__name__)), fontsize=24)
    plt.text(-0.5, 0.7, 'Clustering took {:.2f} s'.format(end_time - start_time), fontsize=14)

In [ ]:
plot_clusters(df_users, cluster.KMeans, (), {'n_clusters':6})